In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from datetime import datetime, timezone


import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import json

from binance.client import Client

from datetime import datetime, timezone, timedelta

In [2]:
import sys
from os.path import expanduser
sys.path.append("{0}/repos/imprint/".format(expanduser("~")))

In [3]:
json_data=open("/Users/lucapozzi/binance_credentials.json").read()
headers = json.loads(json_data)
binance_client = Client(headers['key'], headers['secret'], tld='us')

In [8]:
import unicorn_binance_rest_api
# pip install unicorn-binance-rest-api

binance_rest_client = unicorn_binance_rest_api.BinanceRestApiManager(headers['key'], headers['secret'])

markets = []
data = binance_rest_client.get_all_tickers()
for item in data:
    markets.append(item['symbol'])
    
[market for market in markets if "USDC" in market]
# [market for market in markets if "ADAUS" in market]

['ADAUSDT', 'ADAUSDC']

In [47]:
from unicorn_binance_websocket_api.unicorn_binance_websocket_api_manager import BinanceWebSocketApiManager

binance_websocket_api_manager = BinanceWebSocketApiManager(exchange="binance.com")
binance_websocket_api_manager.create_stream([#'trade', 
                                             'kline_15m'], ['btcusdt', 'bnbbtc', 'ethbtc'], output="UnicornFy")


UUID('276bc089-0e6f-45a1-b34f-3eca4ccb9fdd')

In [49]:
oldest_stream_data_from_stream_buffer = \
binance_websocket_api_manager.pop_stream_data_from_stream_buffer()
oldest_stream_data_from_stream_buffer

{'stream_type': 'ethbtc@kline_15m',
 'event_type': 'kline',
 'event_time': 1626385381631,
 'symbol': 'ETHBTC',
 'kline': {'kline_start_time': 1626384600000,
  'kline_close_time': 1626385499999,
  'symbol': 'ETHBTC',
  'interval': '15m',
  'first_trade_id': False,
  'last_trade_id': False,
  'open_price': '0.06070100',
  'close_price': '0.06065200',
  'high_price': '0.06075200',
  'low_price': '0.06064100',
  'base_volume': '143.34700000',
  'number_of_trades': 445,
  'is_closed': False,
  'quote': '8.70048705',
  'taker_by_base_asset_volume': '69.17600000',
  'taker_by_quote_asset_volume': '4.19865989',
  'ignore': '0'},
 'unicorn_fied': ['binance.com', '0.11.0']}

In [46]:
"result" not in oldest_stream_data_from_stream_buffer

True

In [6]:
import pandas as pd
from datetime import datetime, timezone
def _process_time_stamp(time_stamp):
    return pd.to_datetime(datetime.fromtimestamp(time_stamp / 1000, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S'))

print(_process_time_stamp(1626386656946))
print(_process_time_stamp(1626386659139))

2021-07-15 22:04:16
2021-07-15 22:04:19


In [ ]:
from unicorn_binance_websocket_api.unicorn_binance_websocket_api_manager import BinanceWebSocketApiManager
import time

binance_websocket_api_manager = BinanceWebSocketApiManager(exchange="binance.com")
binance_websocket_api_manager.create_stream(['kline_15m'], ['ethusdt'], output="UnicornFy")

print("Let's GO!!!")

while True:
    stream_data = binance_websocket_api_manager.pop_stream_data_from_stream_buffer()
    # "Warm up the Stream"?
    if stream_data is False:
        time.sleep(0.1)
    if stream_data and ("result" not in stream_data):
        kline = stream_data['kline']
        if kline['is_closed']:
            # Completed kline
            print("[COMPLETE] {0}".format(stream_data))
        else:
            # Process the current price
            print(stream_data)

Let's GO!!!
{'stream_type': 'ethusdt@kline_15m', 'event_type': 'kline', 'event_time': 1626386656946, 'symbol': 'ETHUSDT', 'kline': {'kline_start_time': 1626386400000, 'kline_close_time': 1626387299999, 'symbol': 'ETHUSDT', 'interval': '15m', 'first_trade_id': False, 'last_trade_id': False, 'open_price': '1918.28000000', 'close_price': '1918.39000000', 'high_price': '1921.24000000', 'low_price': '1918.08000000', 'base_volume': '1494.07086000', 'number_of_trades': 1546, 'is_closed': False, 'quote': '2868691.93783500', 'taker_by_base_asset_volume': '911.41055000', 'taker_by_quote_asset_volume': '1750099.82160890', 'ignore': '0'}, 'unicorn_fied': ['binance.com', '0.11.0']}
{'stream_type': 'ethusdt@kline_15m', 'event_type': 'kline', 'event_time': 1626386659139, 'symbol': 'ETHUSDT', 'kline': {'kline_start_time': 1626386400000, 'kline_close_time': 1626387299999, 'symbol': 'ETHUSDT', 'interval': '15m', 'first_trade_id': False, 'last_trade_id': False, 'open_price': '1918.28000000', 'close_price

In [14]:
from trading.utils.data_processing import process_candle, process_candle_from_stream
process_candle_from_stream(json.loads(oldest_stream_data_from_stream_buffer)['data']['k'])

{'open_time': Timestamp('2021-07-15 21:30:00'),
 'open': 0.060701,
 'high': 0.060705,
 'low': 0.060671,
 'close': 0.060689,
 'ohlc4': 0.060691499999999995,
 'volume': 15.98,
 'close_time': Timestamp('2021-07-15 21:44:59'),
 'quote_asset_volume': 0.96982213,
 'n_trades': 43,
 'taker_buy_base': 13.293,
 'taker_buy_quote': 0.80675886}

In [ ]:
from binance import AsyncClient, BinanceSocketManager
def _process_time_stamp(time_stamp):
    return pd.to_datetime(datetime.fromtimestamp(time_stamp/1000, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S'))

symbol = 'BNBBTC'

async def kline_listener(client):
    bm = BinanceSocketManager(client)
    res_count = 0
    async with bm.kline_socket(symbol=symbol, interval=Client.KLINE_INTERVAL_15MINUTE) as stream:
        while True:
            res = await stream.recv()
            res_count += 1
            print(_process_time_stamp(res['E']))
            if res['k']['x']:
                print("Completed kline: {0}".format(res['k']))
            else:
                print("Current Price: {0}".format(res['k']['c']))

# headers['key'], headers['secret'], tld='us'
# client = await AsyncClient.create()
# await kline_listener(client)
await kline_listener(binance_client)

2021-07-14 02:51:12
Current Price: 0.00927480
2021-07-14 02:52:05
Current Price: 0.00927750
2021-07-14 02:53:03
Current Price: 0.00928090
2021-07-14 02:53:59
Current Price: 0.00927760
2021-07-14 02:54:01
Current Price: 0.00927480
2021-07-14 02:54:06
Current Price: 0.00927880
2021-07-14 02:54:08
Current Price: 0.00927880
2021-07-14 02:54:25
Current Price: 0.00927750


In [ ]:

1

In [5]:
candle = binance_client.get_klines(symbol='ETHUSDC', interval=Client.KLINE_INTERVAL_15MINUTE)
datetime.fromtimestamp(candle[-1][6]/1000, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S')


'2021-06-24 02:29:59'

In [46]:
candle = binance_client.get_klines(symbol='ETHUSDC', limit=2, interval=Client.KLINE_INTERVAL_15MINUTE)
datetime.fromtimestamp(candle[-2][6]/1000, #tz=timezone.utc
                      ).strftime('%Y-%m-%d %H:%M:%S')

'2021-06-23 20:29:59'

In [ ]:
candle[-1][4]
binance_client.get_all_tickers('ETHUSDC')